# Ejercicios Avanzados de Pandas

Este notebook contiene ejercicios elaborados para practicar operaciones complejas con pandas. Cada ejercicio requiere combinar múltiples técnicas y pensamiento analítico.

**Instrucciones:**
- Lee cada enunciado cuidadosamente
- Los ejercicios requieren combinar varias operaciones de pandas
- Intenta resolver cada problema paso a paso
- Algunos ejercicios tienen múltiples soluciones válidas

In [2]:
# Importar librerías
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# Cargar los datos
data = pd.read_csv("material/data.csv")
data.head(10)

,order_id,fecha,order_customer_id,product_id,unit_price,quantity,price,product_family,latitude,longitude
0,1016330781887,2023-05-29,6451824076,IAM925P00XXZPUDIA,19.0,8,152.0,Pulsera,37.788617,-3.790215
1,1095376438463,2023-07-16,664062345899,OTS925P00XXZPUDOT,19.0,6,114.0,Pulsera,40.299542,-3.926774
2,1047641588927,2023-06-17,642133246635,UND925P00XXZCOMUN,25.0,5,125.0,Collar,39.651927,-0.411277
3,1181322118257,2023-08-29,714672128644,OTS925A00XXZPUDOT,19.0,4,76.0,Pulsera,39.456511,-0.346203
4,1017492866239,2023-05-30,196440539724,IMO925P00XXZTOMIM,25.0,3,75.0,Tobillera,37.394171,-5.957857
5,480728843265,2023-03-27,226613674572,UDL925P00XXVPUSUD,25.0,3,75.0,Pulsera,38.984944,-3.927849
6,457358836737,2023-03-10,244904182348,UND925P00XXZCOMUN,25.0,3,75.0,Collar,40.604510,-4.339488
7,1001207956671,2023-05-21,621451002539,OTS925P00XXZTODOT,19.0,3,57.0,Tobillera,40.804126,0.516218
8,1057963181247,2023-06-23,647359906475,UND925P00XXZCOMUN,25.0,3,75.0,Collar,41.361692,2.129037
9,1095376438463,2023-07-16,664062345899,OTS925P00XXZPUDOT,19.0,3,57.0,Pulsera,40.299542,-3.926774


---

### Ejercicio 1: Clientes VIP
Identifica a los clientes VIP basándote en los siguientes criterios:
- Han realizado al menos 3 órdenes diferentes
- Su gasto total supera los 100€

Crea un DataFrame con:
- `order_customer_id`
- Número de órdenes
- Gasto total
- Una columna booleana `es_vip` que indique si cumple los 2 criterios

Ordena el resultado por gasto total descendente.

In [4]:
# SOLUCION

# Agrupar por cliente y calcular métricas
clientes = data.groupby('order_customer_id', as_index = False).agg({
    'order_id': 'nunique',  # Número de órdenes diferentes
    'price': 'sum' # Gasto total 
})

clientes.columns = ['order_customer_id', 'num_ordenes', 'gasto_total']

# Crear columna es_vip con los 2 criterios
clientes['es_vip'] = (
    (clientes['num_ordenes'] >= 3) & 
    (clientes['gasto_total'] > 100) 
)

# Ordenar por gasto total descendente
clientes_vip = clientes.sort_values('gasto_total', ascending=False)

print(f"Total de clientes VIP: {clientes_vip['es_vip'].sum()}")
print(f"\nPrimeros 10 clientes:")
clientes_vip.head(10)

Total de clientes VIP: 53

Primeros 10 clientes:


,order_customer_id,num_ordenes,gasto_total,es_vip
93,5753058444,4,432.70,True
498,254713512524,6,431.00,True
419,243532710476,5,425.25,True
895,626415125163,1,346.00,False
38,4594553932,8,346.00,True
218,7012581324,5,326.80,True
635,600548623019,4,308.75,True
878,624689627819,5,259.25,True
1129,663846044331,4,258.15,True
1134,664062345899,2,247.00,False


### Ejercicio 2: Análisis de Cestas de Compra
Para cada orden (`order_id`), calcula:
- Número total de productos en la orden
- Número de unidades totales compradas
- Importe total de la orden
- Número de familias de productos diferentes en la orden
- gasto medio por order

Luego identifica:
- Las 5 órdenes con mayor diversidad de productos (más familias diferentes)
- Las 5 órdenes con mayor importe total
- ¿Hay solapamiento entre estas dos listas?

In [5]:
# Ejercicio 2: Análisis de Cestas de Compra

# Calcular métricas por orden
compras = data.groupby('order_id', as_index= False).agg({
    'product_id': 'nunique',  # Número de productos
    'quantity': 'sum',  # Unidades totales
    'price': ['sum', "mean"],  # Importe total y gasto medio
    'product_family': 'nunique',  # Familias diferentes
})

compras.columns = ['order_id', 'num_productos', 'unidades_totales', 'importe_total', 'gasto_medio', 'num_familias' ]

print("=== Análisis de Cestas de Compra ===\n")

# Top 5 órdenes con mayor diversidad
print("Top 5 órdenes con mayor diversidad de productos:")
top_diversidad = compras.sort_values('num_familias', ascending=False).head(5)
print(top_diversidad)
print("--------")
# Top 5 órdenes con mayor importe
print("Top 5 órdenes con mayor importe total:")
top_importe = compras.sort_values('importe_total', ascending=False).head(5)
print(top_importe)
print("--------")
# Verificar solapamiento
ordenes_diversidad = set(top_diversidad['order_id'])
ordenes_importe = set(top_importe['order_id'])
solapamiento = ordenes_diversidad.intersection(ordenes_importe)
print(f"Órdenes que aparecen en ambas listas: {len(solapamiento)}")
if solapamiento:
    print(f"IDs: {solapamiento}")
else:
    print("No hay solapamiento entre ambas listas")

=== Análisis de Cestas de Compra ===

Top 5 órdenes con mayor diversidad de productos:
           order_id  num_productos  unidades_totales  importe_total  \
1132  1069883917503              9                 9         187.25   
1169  1076911670463              6                 6         135.00   
1038  1047824303295              4                 4          88.00   
380    933853136063              4                 4         104.00   
194    479898305537              4                 4         114.00   

      gasto_medio  num_familias  
1132    20.805556             5  
1169    22.500000             5  
1038    22.000000             4  
380     26.000000             4  
194     28.500000             4  
--------
Top 5 órdenes con mayor importe total:
           order_id  num_productos  unidades_totales  importe_total  \
788   1012895974591              2                 4         346.00   
669    992577586367              1                 1         199.00   
1132  1069883917503  

### Ejercicio 3: Patrón de Compra Recurrente
Identifica qué clientes compran regularmente (tienen órdenes en al menos 3 fechas diferentes) y cuál es su familia de productos favorita (la que más compran en términos de cantidad total).

Crea un DataFrame que muestre:
- `order_customer_id`
- Número de días diferentes en que ha comprado
- Familia de productos favorita
- Cantidad total comprada de esa familia
- Porcentaje que representa esa familia sobre sus compras totales

In [ ]:
# Obtenemos numero de fechas distintas y cantidad total por cliente
fechas_y_cantidad_total = data.groupby("order_customer_id", as_index = False).agg(
    {
        "fecha": "nunique", 
        "quantity": "sum"
    }
    )
fechas_y_cantidad_total.columns = ["order_customer_id", "num_fechas", "cantidad_total"]

# Obtenemos la familia de productos favorita por cliente
compras_por_familia = data.groupby(["order_customer_id", "product_family"], as_index = False)["quantity"].sum()
familia_favorita = compras_por_familia.loc[compras_por_familia.groupby("order_customer_id")["quantity"].idxmax()]
familia_favorita.columns = ["order_customer_id", "familia_favorita", "cantidad_familia_favorita"]

# Unimos ambos dataframes y calculamos el porcentaje de la familia favorita
resultado = fechas_y_cantidad_total.merge(familia_favorita, on = "order_customer_id")
resultado["porcentaje_familia_favorita"] = 100 * resultado["cantidad_familia_favorita"] / resultado["cantidad_total"]
resultado.drop(columns = ["cantidad_total"], inplace = True)

print("=== Análisis de Clientes ===\n")
print("Primeros 10 clientes analizados:")
print(resultado.head(10))

# Clientes que han comprado en más de 2 fechas distintas
print(f"\nNúmero de clientes que han comprado en más de 2 fechas distintas: ")
print(resultado[resultado["num_fechas"] > 2])


=== Análisis de Clientes ===

Primeros 10 clientes analizados:
   order_customer_id  num_fechas familia_favorita  cantidad_familia_favorita  \
0         4438242304           3           Collar                          2   
1         4438303040           1           Collar                          2   
2         4438358016           1        Tobillera                          1   
3         4438388032           1           Anillo                          1   
4         4438411456           1           Collar                          3   
5         4438414656           1          Pulsera                          2   
6         4438568448           1       Pendientes                          3   
7         4438581888           2           Collar                          2   
8         4438603712           1           Collar                          1   
9         4438679360           1           Collar                          3   

   porcentaje_familia_favorita  
0                    40

### Ejercicio 4: Tendencia de Ventas
Convierte la columna `fecha` a datetime y realiza el siguiente análisis:

1. Calcula las ventas diarias totales (suma de `price`)
2. Añade una columna con la diferencia porcentual respecto al día anterior
3. Identifica los 3 días con mayor crecimiento porcentual
4. Identifica los 3 días con mayor caída porcentual


In [7]:
# Ejercicio 4: Tendencia de Ventas

# Convertir fecha a datetime (si no lo está ya)
data['fecha'] = pd.to_datetime(data['fecha'])

# Calcular ventas diarias
ventas_diarias = data.groupby('fecha', as_index = False)['price'].sum()
ventas_diarias.columns = ['fecha', 'ventas_totales']
ventas_diarias = ventas_diarias.sort_values('fecha')

# Calcular diferencia porcentual respecto al día anterior
ventas_diarias['ventas_dia_anterior'] = ventas_diarias['ventas_totales'].shift(1)

ventas_diarias['diferencia_porcentual'] = (
    100 * (ventas_diarias['ventas_totales'] - ventas_diarias['ventas_dia_anterior']) / ventas_diarias['ventas_dia_anterior']
).round(2)
ventas_diarias

# Top 3 días con mayor crecimiento
print("Top 3 días con MAYOR crecimiento porcentual:")
top_crecimiento = ventas_diarias.sort_values('diferencia_porcentual', ascending=False).head(3)
print(top_crecimiento)
print("--------")

# Top 3 días con mayor caída
print("Top 3 días con MAYOR caída porcentual:")
top_caida = ventas_diarias.sort_values('diferencia_porcentual', ascending=True).head(3)
print(top_caida)


Top 3 días con MAYOR crecimiento porcentual:
         fecha  ventas_totales  ventas_dia_anterior  diferencia_porcentual
64  2023-05-06          985.00                83.00                1086.75
148 2023-07-29          485.25                46.75                 937.97
3   2023-03-06          723.00                75.00                 864.00
--------
Top 3 días con MAYOR caída porcentual:
         fecha  ventas_totales  ventas_dia_anterior  diferencia_porcentual
174 2023-08-24           39.25               590.15                 -93.35
54  2023-04-26          119.00               750.00                 -84.13
27  2023-03-30           79.00               452.00                 -82.52


### Ejercicio 5: Análisis de Fin de Semana vs Entre Semana
Compara el comportamiento de compra entre fin de semana (sábado y domingo) y entre semana:

Para cada segmento calcula:
- Número de órdenes únicas
- Ticket medio (gasto medio por orden)
- Productos por orden (media)
- Familia de productos más vendida
- Porcentaje de ventas de cada familia

Presenta los resultados en un formato que permita comparar fácilmente ambos segmentos.

In [8]:
# Ejercicio 5: Análisis Fin de Semana vs Entre Semana

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# Crear columna día de la semana (0=lunes, 6=domingo)
data['is_weekend'] = data['fecha'].dt.day_name().isin(["Sunday", "Saturday"])

compras = data.groupby("is_weekend", as_index = False).agg({
    'order_id': 'nunique',  # Número de órdenes únicas
    'price': 'sum',        # Ventas totales
    'product_id': 'nunique',  # Número de productos vendidos
})

compras.columns = ['is_weekend', 'num_ordenes', 'gasto_total', 'num_productos_vendidos']
compras["gasto_medio"]= compras["gasto_total"] / compras["num_ordenes"]
compras["productos_por_orden"] = compras["num_productos_vendidos"] / compras["num_ordenes"]



# Obtenemos la familia de productos favorita 
compras_por_familia = data.groupby(["is_weekend", "product_family"], as_index = False)["quantity"].sum()
familia_favorita = compras_por_familia.iloc[compras_por_familia.groupby("is_weekend")["quantity"].idxmax()]
familia_favorita.columns = ["is_weekend", "familia_favorita", "cantidad_familia_favorita"]

compras = compras.merge(familia_favorita, on = "is_weekend")
compras["porcentaje_familia_favorita"] = 100 * compras["cantidad_familia_favorita"] / compras["num_productos_vendidos"]
compras.drop(columns = ["cantidad_familia_favorita", "num_productos_vendidos"], inplace = True)
print(compras)


   is_weekend  num_ordenes  gasto_total  gasto_medio  productos_por_orden  \
0       False         1129     53863.47    47.709008             0.234721   
1        True          528     26189.40    49.601136             0.409091   

  familia_favorita  porcentaje_familia_favorita  
0           Collar                   289.433962  
1           Collar                   186.111111  


### Ejercicio 6: Cohort Analysis Simplificado
Realiza un análisis de cohortes básico:

1. Para cada cliente, identifica la fecha de su primera compra (fecha de adquisición)
2. Agrupa a los clientes por semana de adquisición
3. Para cada cohorte, calcula:
   - Número de clientes en la cohorte
   - Gasto total de la cohorte
   - Gasto medio por cliente
   - Número medio de órdenes por cliente
   
Ordena las cohortes de más antigua a más reciente.

In [9]:
# Ejercicio 6: Cohort Analysis Simplificado

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])
data["fecha_week_start"] = data['fecha'].dt.to_period('W').dt.start_time


# Calcular agregaciones por cliente
compras = data.groupby("order_customer_id", as_index = False).agg({
    'fecha_week_start': 'min',
    'price': 'sum',
    'order_id': 'nunique' 
})
compras.columns = ['order_customer_id', 'fecha_primera_compra', 'gasto_total', 'num_ordenes']

print(compras.sort_values("fecha_primera_compra"))

# Calcular agregaciones por cohorte
compras = compras.groupby("fecha_primera_compra", as_index = False).agg({
    'order_customer_id': 'nunique',
    'gasto_total': ['sum', 'mean'],
    'num_ordenes': 'mean'
})
compras.columns = ['fecha_primera_compra', 'num_clientes', 'gasto_total', 'gasto_medio', 'ordenes_medias']

print(compras.sort_values("fecha_primera_compra"))



      order_customer_id fecha_primera_compra  gasto_total  num_ordenes
143          6618401292           2023-02-27         89.0            1
261         55872635468           2023-02-27        120.0            3
349        206959821388           2023-02-27         89.0            1
400        241611129420           2023-02-27         75.0            1
284        105953837644           2023-02-27         83.0            1
...                 ...                  ...          ...          ...
177          6893212300           2023-10-02         29.0            1
816        616726282923           2023-10-02         19.0            1
101          5987654092           2023-10-02         29.0            1
1187       672710935172           2023-10-02         54.0            1
1286       713419604612           2023-10-02         58.0            1

[1314 rows x 4 columns]
   fecha_primera_compra  num_clientes  gasto_total  gasto_medio  \
0            2023-02-27            13       924.00    71


### Ejercicio 7: Elasticidad de Cantidad
Analiza cómo varía la cantidad comprada según el precio unitario:

1. Crea rangos de precio unitario (bins): 0-20, 20-40, 40-60, 60+
2. Para cada rango y cada familia de productos, calcula:
   - Cantidad media comprada
   - Número de transacciones
3. Crea una tabla pivote que muestre la cantidad media comprada con:
   - Filas: familias de productos
   - Columnas: rangos de precio
4. ¿Qué familia de productos es menos sensible al precio (mantiene cantidades altas incluso a precios altos)?

In [10]:
# Ejercicio 7: Elasticidad de Cantidad

# Crear rangos de precio unitario
data['rango_precio'] = pd.cut(data['unit_price'], 
    bins=[0, 20, 40, 60, float('inf')],
    labels=['0-20', '20-40', '40-60', '60+'],
    right=False
)

# Calcular métricas por rango de precio y familia
elasticidad = data.groupby(['product_family', 'rango_precio']).agg({
    'product_id': 'nunique',  # Número de productos
    'quantity': 'sum',  # Cantidad total comprada
    'order_id': 'nunique',  # Número de transacciones
}).reset_index()

elasticidad.columns = ['familia', 'rango_precio', 'numero_productos', 'cantidad_total', 'num_transacciones']
elasticidad["cantidad_media"] = elasticidad["cantidad_total"] / elasticidad["numero_productos"]
# Crear tabla pivote: cantidad media con familias en filas y rangos en columnas
tabla_pivote = elasticidad.pivot_table(
    index='familia',
    columns='rango_precio',
    values='cantidad_media',
    fill_value=0
).round(2)

print("=== Tabla Pivote: Cantidad Media Comprada ===")
print("Filas: Familias de productos | Columnas: Rangos de precio\n")
print(tabla_pivote)
print("\n")


=== Tabla Pivote: Cantidad Media Comprada ===
Filas: Familias de productos | Columnas: Rangos de precio

rango_precio   0-20  20-40  40-60  60+
familia                               
Anillo         6.12   3.37   0.00  0.0
Collar        13.08  18.62   4.31  4.0
Pendientes     7.10   9.96   3.00  1.0
Pulsera        7.76   7.32   7.00  0.0
Tobillera     12.67  25.38   0.00  0.0




/tmp/ipykernel_865681/2020987983.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elasticidad = data.groupby(['product_family', 'rango_precio']).agg({
/tmp/ipykernel_865681/2020987983.py:20: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tabla_pivote = elasticidad.pivot_table(


### Ejercicio 8: Análisis de Margen de Contribución
Simula el coste del producto (como mucho 40% del `unit_price`). Calcula para cada familia de productos:

1. Margen unitario promedio (unit_price - coste)
2. Margen total (suma de todos los márgenes de las ventas)
3. Contribución al margen total del negocio (porcentaje)

Ordena por contribución al margen descendente. ¿Cuál es la familia más rentable?

In [ ]:
# Ejercicio 8: Análisis de Margen de Contribución

# Simular coste del producto (40% del unit_price)
data['coste'] = np.random.uniform(0.1, data['unit_price'] * 0.4)

# Calcular margen unitario
data['margen_unitario'] = data['unit_price'] - data['coste']

# Calcular margen por transacción (margen_unitario * quantity)
data['margen_transaccion'] = data['margen_unitario'] * data['quantity']

# Análisis por familia de productos
margen_familia = data.groupby('product_family').agg({
    'product_id': 'nunique',  # Número de productos
    'margen_unitario': 'sum',  # Suma margen unitario 
    'margen_transaccion': 'sum',  # Margen total
}).reset_index()

margen_familia.columns = ['familia','numero_productos', 'margen_unitario_sum', 'margen_total']
margen_familia['margen_unitario_promedio'] = (
    margen_familia['margen_unitario_sum'] / margen_familia['numero_productos']
)

# Calcular contribución al margen total del negocio
margen_total_negocio = margen_familia['margen_total'].sum()
margen_familia['contribucion_porcentaje'] = (
    margen_familia['margen_total'] / margen_total_negocio * 100
).round(2)

# Ordenar por contribución descendente
margen_familia = margen_familia.sort_values('contribucion_porcentaje', ascending=False)

print("=== Análisis de Margen de Contribución por Familia ===\n")
print(margen_familia)
print(f"\n{'='*60}")

=== Análisis de Margen de Contribución por Familia ===

      familia  margen_unitario_promedio  margen_total  contribucion_porcentaje
1      Collar                 22.614425  26400.824733                    41.13
2  Pendientes                 20.172442  13538.783472                    21.09
3     Pulsera                 19.796916   9473.357783                    14.76
0      Anillo                 18.381840   9344.894488                    14.56
4   Tobillera                 19.520266   5425.485416                     8.45



### Ejercicio 9: Segmentación RFM (Recency, Frequency, Monetary)
Crea una segmentación RFM de clientes:

1. **Recency**: Días desde la última compra (usa la fecha más reciente del dataset como referencia)
2. **Frequency**: Número de órdenes del cliente
3. **Monetary**: Gasto total del cliente

Para cada métrica:
- Asigna un score de 1 a 4 usando cuartiles (1=peor, 4=mejor)
- Para Recency, invierte la lógica (menor recency = mejor = score más alto)

Crea una columna `RFM_Score` concatenando los tres scores (ej: "444" = cliente excelente).

Finalmente, clasifica a los clientes en segmentos de tu ínteres.

¿Cuántos clientes hay en cada segmento?

In [11]:
# Ejercicio 9: Segmentación RFM (Recency, Frequency, Monetary)

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# Fecha de referencia (fecha más reciente del dataset)
fecha_referencia = data['fecha'].max()

# Calcular métricas RFM por cliente
rfm = data.groupby('order_customer_id', as_index = False).agg({
    'fecha': lambda x: (fecha_referencia - x.max()).days,  # Recency: días desde última compra
    'order_id': 'nunique',  # Frequency: número de órdenes
    'price': 'sum'  # Monetary: gasto total
})

rfm.columns = ['order_customer_id', 'recency', 'frequency', 'monetary']

# Asignar scores usando cuartiles (1=peor, 4=mejor)
# Para Recency: menor es mejor (invertir lógica)
rfm['R_score'] = pd.qcut(rfm['recency'], q=4, labels=[4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['frequency'].rank(method='first'), q=4, labels=[1, 2, 3, 4])
rfm['M_score'] = pd.qcut(rfm['monetary'].rank(method='first'), q=4, labels=[1, 2, 3, 4])

# # Convertir scores a string y concatenar
rfm['RFM_Score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)
rfm

# Clasificar en segmentos
def clasificar_segmento(row):
    r, f, m = int(row['R_score']), int(row['F_score']), int(row['M_score'])
    
    # Clientes excelentes (Champions)
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    # Clientes leales (Loyal)
    elif f >= 3 and m >= 3:
        return 'Loyal Customers'
    # Clientes potenciales (Potential)
    elif r >= 3 and f <= 2 and m <= 2:
        return 'Potential Loyalists'
    # Nuevos clientes (New)
    elif r >= 4 and f <= 2:
        return 'New Customers'
    # En riesgo (At Risk)
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    # Perdidos (Lost)
    elif r <= 2 and f <= 2:
        return 'Lost'
    else:
        return 'Others'

rfm['segmento'] = rfm.apply(clasificar_segmento, axis=1)

print("=== Segmentación RFM de Clientes ===\n")

# Contar clientes por segmento
rfm.groupby('segmento')["order_customer_id"].count()

=== Segmentación RFM de Clientes ===



segmento
Champions               94
Lost                   530
Loyal Customers        302
New Customers           22
Others                 287
Potential Loyalists     79
Name: order_customer_id, dtype: int64